<a href="https://colab.research.google.com/github/Fentahun60/Fentahun/blob/main/Safewater_access.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to your CSV file
csv_file_path = '/content/drive/MyDrive/Safewater_CSV_all_new.csv'

# Read the CSV file into a DataFrame
try:
    df = pd.read_csv(csv_file_path)
    # Display the head of the DataFrame
    print("DataFrame head:")
    print(df.head())
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found. Please ensure the path is correct and your Drive is mounted.")
except Exception as e:
    print(f"An error occurred while reading the CSV: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DataFrame head:
        case_id country_code_phase  cluster_no  household_no  \
0      00010001                AO7           1             1   
1      00010002                AO7           1             2   
2      00010003                AO7           1             3   
3      00010004                AO7           1             4   
4      00010005                AO7           1             5   

   hh_sample_weight  interview_year  Number_household_members  \
0           1203191            2016                         7   
1           1203191            2016                         9   
2           1203191            2016                         7   
3           1203191            2016                         7   
4           1203191            2016                         2   

   Number_children_under5 Residence  total_adults  ... Has_electricity  \
0    

In [67]:
df.columns

Index(['case_id', 'country_code_phase', 'cluster_no', 'household_no',
       'hh_sample_weight', 'interview_year', 'Number_household_members',
       'Number_children_under5', 'Residence', 'total_adults',
       'source_drinking_water', 'distance_to_water_source', 'toilet_type',
       'Has_electricity', 'Has_radio', 'Has_television', 'Has_refrigerator',
       'Numer_sleeping_rooms', 'household_head_sex', 'Household_head_age',
       'Household_wealth_index', 'Household_education_level',
       'Household_head_marital_status'],
      dtype='object')

In [68]:
df = df[df['country_code_phase'] != 'AO7']
display(df.head())

,case_id,country_code_phase,cluster_no,household_no,hh_sample_weight,interview_year,Number_household_members,Number_children_under5,Residence,total_adults,...,Has_electricity,Has_radio,Has_television,Has_refrigerator,Numer_sleeping_rooms,household_head_sex,Household_head_age,Household_wealth_index,Household_education_level,Household_head_marital_status
16109,4 3,BF8,4,3,1203283,2021,4,1,rural,1.0,...,no,no,no,no,2.0,male,32,poorest,primary,married
16110,4 8,BF8,4,8,1203283,2021,7,2,rural,0.0,...,no,yes,no,no,2.0,male,34,middle,"no education, preschool",married
16111,4 13,BF8,4,13,1203283,2021,9,2,rural,1.0,...,no,no,no,no,4.0,male,34,poorest,secondary,married
16112,4 17,BF8,4,17,1203283,2021,8,1,rural,0.0,...,no,yes,no,no,2.0,male,41,poorer,"no education, preschool",married
16113,4 22,BF8,4,22,1203283,2021,6,1,rural,0.0,...,no,no,no,no,2.0,male,66,poorest,"no education, preschool",married


In [69]:
missing_values_percentage = df.isnull().sum() / len(df) * 100
print("Percentage of missing values per column:")
print(missing_values_percentage[missing_values_percentage > 0].sort_values(ascending=False))

Percentage of missing values per column:
total_adults                     3.664047
Household_head_marital_status    2.012588
Numer_sleeping_rooms             0.048680
Household_education_level        0.028736
toilet_type                      0.018442
Has_refrigerator                 0.006648
distance_to_water_source         0.005576
source_drinking_water            0.004932
Has_electricity                  0.004932
Has_television                   0.004503
Has_radio                        0.003217
Household_head_age               0.001716
dtype: float64


In [70]:
print(df['country_code_phase'].value_counts())

country_code_phase
NG8    40047
KE8    37911
MW7    26361
CD8    26347
MD7    20510
UG7    19588
GH8    17933
TD6    17233
AO8    16243
BU7    15977
TZ8    15705
CI8    14766
MZ8    14250
BJ7    14156
SL7    13399
BF8    13251
RW7    12949
ZM7    12831
GA7    11781
CM7    11710
MR7    11658
ZA7    11083
ZW7    10534
LS8     9810
ML8     9667
LB7     9068
ET7     8663
SN8     8423
GN7     7912
GM7     6549
Name: count, dtype: int64


In [71]:
selected_columns = [
    'country_code_phase',
    'interview_year',
    'Number_household_members',
    'Number_children_under5',
    'Residence',
    'total_adults',
    'source_drinking_water',
    'distance_to_water_source',
    'toilet_type',
    'Has_electricity',
    'Has_radio',
    'Has_television',
    'Has_refrigerator',
    'Numer_sleeping_rooms',
    'household_head_sex',
    'Household_head_age',
    'Household_wealth_index',
    'Household_education_level',
    'Household_head_marital_status'
]

df_sel = df[selected_columns]
display(df_sel.head())

,country_code_phase,interview_year,Number_household_members,Number_children_under5,Residence,total_adults,source_drinking_water,distance_to_water_source,toilet_type,Has_electricity,Has_radio,Has_television,Has_refrigerator,Numer_sleeping_rooms,household_head_sex,Household_head_age,Household_wealth_index,Household_education_level,Household_head_marital_status
16109,BF8,2021,4,1,rural,1.0,unprotected well,20,no facility/outdoors/bush,no,no,no,no,2.0,male,32,poorest,primary,married
16110,BF8,2021,7,2,rural,0.0,unprotected well,30,no facility/outdoors/bush,no,yes,no,no,2.0,male,34,middle,"no education, preschool",married
16111,BF8,2021,9,2,rural,1.0,unprotected well,5,no facility/outdoors/bush,no,no,no,no,4.0,male,34,poorest,secondary,married
16112,BF8,2021,8,1,rural,0.0,22,5,indoors: latrine to septic tank,no,yes,no,no,2.0,male,41,poorer,"no education, preschool",married
16113,BF8,2021,6,1,rural,0.0,22,10,no facility/outdoors/bush,no,no,no,no,2.0,male,66,poorest,"no education, preschool",married


In [72]:
country_mapping = {
    'AO8': 'Angola', 'BF8': 'Burkina Faso', 'BJ7': 'Benin', 'BU7': 'Burundi',
    'CD8': 'Congo Democratic Republic', 'CI8': 'Côte d’Ivoire', 'CM7': 'Cameroon',
    'ET7': 'Ethiopia', 'GA7': 'Gabon', 'GH8': 'Ghana', 'GM7': 'Gambia',
    'GN7': 'Guinea', 'KE8': 'Kenya', 'LB7': 'Liberia', 'LS8': 'Lesotho',
    'MD7': 'Madagascar', 'ML8': 'Mali', 'MR7': 'Mauritania', 'MW7': 'Malawi',
    'MZ8': 'Mozambique', 'NG8': 'Nigeria', 'RW7': 'Rwanda', 'SL7': 'Sierra Leone',
    'SN8': 'Senegal', 'TD6': 'Chad', 'TZ8': 'Tanzania', 'UG7': 'Uganda',
    'ZA7': 'South Africa', 'ZM7': 'Zambia', 'ZW7': 'Zimbabwe'
}

df_sel['country_code_phase'] = df_sel['country_code_phase'].map(country_mapping)
display(df_sel.head())

/tmp/ipython-input-2845534014.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['country_code_phase'] = df_sel['country_code_phase'].map(country_mapping)


,country_code_phase,interview_year,Number_household_members,Number_children_under5,Residence,total_adults,source_drinking_water,distance_to_water_source,toilet_type,Has_electricity,Has_radio,Has_television,Has_refrigerator,Numer_sleeping_rooms,household_head_sex,Household_head_age,Household_wealth_index,Household_education_level,Household_head_marital_status
16109,Burkina Faso,2021,4,1,rural,1.0,unprotected well,20,no facility/outdoors/bush,no,no,no,no,2.0,male,32,poorest,primary,married
16110,Burkina Faso,2021,7,2,rural,0.0,unprotected well,30,no facility/outdoors/bush,no,yes,no,no,2.0,male,34,middle,"no education, preschool",married
16111,Burkina Faso,2021,9,2,rural,1.0,unprotected well,5,no facility/outdoors/bush,no,no,no,no,4.0,male,34,poorest,secondary,married
16112,Burkina Faso,2021,8,1,rural,0.0,22,5,indoors: latrine to septic tank,no,yes,no,no,2.0,male,41,poorer,"no education, preschool",married
16113,Burkina Faso,2021,6,1,rural,0.0,22,10,no facility/outdoors/bush,no,no,no,no,2.0,male,66,poorest,"no education, preschool",married


In [73]:
print(df_sel['country_code_phase'].value_counts())

country_code_phase
Nigeria                      40047
Kenya                        37911
Malawi                       26361
Congo Democratic Republic    26347
Madagascar                   20510
Uganda                       19588
Ghana                        17933
Chad                         17233
Angola                       16243
Burundi                      15977
Tanzania                     15705
Côte d’Ivoire                14766
Mozambique                   14250
Benin                        14156
Sierra Leone                 13399
Burkina Faso                 13251
Rwanda                       12949
Zambia                       12831
Gabon                        11781
Cameroon                     11710
Mauritania                   11658
South Africa                 11083
Zimbabwe                     10534
Lesotho                       9810
Mali                          9667
Liberia                       9068
Ethiopia                      8663
Senegal                       8423
G

In [74]:
print(df_sel['country_code_phase'].unique())

['Burkina Faso' 'Benin' 'Burundi' 'Congo Democratic Republic'
 'Côte d’Ivoire' 'Cameroon' 'Ethiopia' 'Gabon' 'Ghana' 'Gambia' 'Guinea'
 'Kenya' 'Liberia' 'Lesotho' 'Madagascar' 'Mali' 'Mauritania' 'Malawi'
 'Mozambique' 'Nigeria' 'Rwanda' 'Sierra Leone' 'Senegal' 'Chad'
 'Tanzania' 'Uganda' 'South Africa' 'Zambia' 'Zimbabwe' 'Angola']


In [75]:
print(df_sel['source_drinking_water'].unique())

['unprotected well' '22' '15' '21' 'unprotected spring' 'protected well'
 'piped to yard/plot' '72' 'piped into dwelling' 'public fountain'
 "piped to neighbour's house" 'bottled water'
 'lake/pond/river/channel/irrigation channel' 'rainwater'
 'protected spring' '16' 'cart with small tank' 'tanker truck' 'other'
 '92' '82' nan '73' 'motorcycle with three wheels' 'borehole with pump']


In [76]:
improved_sources = [
    'piped into dwelling',
    'piped to yard/plot',
    'public fountain', # Assuming 'public tap/standpipe'
    "piped to neighbour's house", # Assuming 'piped to neighbor'
    'borehole with pump', # Assuming 'tube well or borehole'
    'protected well',
    'protected spring',
    'rainwater',
    'tanker truck',
    'cart with small tank',
    'bottled water',
    '21' # Numerical code for tube well or borehole
]

df_sel['source_drinking_water'] = df_sel['source_drinking_water'].apply(lambda x: 'Improved' if x in improved_sources else 'Unimproved')

# Display the new column and its value counts to verify
print(df_sel['source_drinking_water'].value_counts())
display(df_sel.head())

source_drinking_water
Improved      327047
Unimproved    139268
Name: count, dtype: int64


/tmp/ipython-input-2330485941.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['source_drinking_water'] = df_sel['source_drinking_water'].apply(lambda x: 'Improved' if x in improved_sources else 'Unimproved')


,country_code_phase,interview_year,Number_household_members,Number_children_under5,Residence,total_adults,source_drinking_water,distance_to_water_source,toilet_type,Has_electricity,Has_radio,Has_television,Has_refrigerator,Numer_sleeping_rooms,household_head_sex,Household_head_age,Household_wealth_index,Household_education_level,Household_head_marital_status
16109,Burkina Faso,2021,4,1,rural,1.0,Unimproved,20,no facility/outdoors/bush,no,no,no,no,2.0,male,32,poorest,primary,married
16110,Burkina Faso,2021,7,2,rural,0.0,Unimproved,30,no facility/outdoors/bush,no,yes,no,no,2.0,male,34,middle,"no education, preschool",married
16111,Burkina Faso,2021,9,2,rural,1.0,Unimproved,5,no facility/outdoors/bush,no,no,no,no,4.0,male,34,poorest,secondary,married
16112,Burkina Faso,2021,8,1,rural,0.0,Unimproved,5,indoors: latrine to septic tank,no,yes,no,no,2.0,male,41,poorer,"no education, preschool",married
16113,Burkina Faso,2021,6,1,rural,0.0,Unimproved,10,no facility/outdoors/bush,no,no,no,no,2.0,male,66,poorest,"no education, preschool",married


In [77]:
print(df_sel['toilet_type'].unique())

['no facility/outdoors/bush' 'indoors: latrine to septic tank' '23' '41'
 '43' 'indoors: latrine to piped public system' 'other'
 'indoors: flush to piped public system' 'indoors: flush to septic tank'
 'indoors: flush to open pit (ditch or river)'
 'inside yard: flush to septic tank'
 'inside yard: flush to piped public system'
 'bucket/potty/other container'
 'inside yard: flush to open pit (ditch or river)' '52' '54' '53' '51' nan
 'inside yard: latrine to piped public system' '44']


In [78]:
import numpy as np

def categorize_toilet_type(x):
    if pd.isna(x):
        return np.nan
    x = x.lower().strip()

    # Flush to piped sewer system (11)
    if 'flush' in x and 'piped public system' in x:
        return 11

    # Flush to septic tank (12)
    elif 'flush' in x and 'septic tank' in x:
        return 12

    # Flush to pit latrine (13)
    elif 'flush' in x and 'pit' in x:
        return 13

    # Flush - don’t know where (15)
    elif 'flush' in x and all(k not in x for k in ['piped', 'septic', 'pit']):
        return 15

    # Improved pit latrine (VIP or with slab) (21, 22)
    elif 'vip' in x:
        return 21
    elif 'with slab' in x:
        return 22

    # Composting toilet (41)
    elif 'compost' in x:
        return 41

    # Otherwise unimproved
    else:
        return 'unimproved'

# Apply the function
df_sel['toilet_types'] = df_sel['toilet_type'].apply(categorize_toilet_type)


/tmp/ipython-input-236804174.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['toilet_types'] = df_sel['toilet_type'].apply(categorize_toilet_type)


In [79]:
df_sel.head()

,country_code_phase,interview_year,Number_household_members,Number_children_under5,Residence,total_adults,source_drinking_water,distance_to_water_source,toilet_type,Has_electricity,Has_radio,Has_television,Has_refrigerator,Numer_sleeping_rooms,household_head_sex,Household_head_age,Household_wealth_index,Household_education_level,Household_head_marital_status,toilet_types
16109,Burkina Faso,2021,4,1,rural,1.0,Unimproved,20,no facility/outdoors/bush,no,no,no,no,2.0,male,32,poorest,primary,married,unimproved
16110,Burkina Faso,2021,7,2,rural,0.0,Unimproved,30,no facility/outdoors/bush,no,yes,no,no,2.0,male,34,middle,"no education, preschool",married,unimproved
16111,Burkina Faso,2021,9,2,rural,1.0,Unimproved,5,no facility/outdoors/bush,no,no,no,no,4.0,male,34,poorest,secondary,married,unimproved
16112,Burkina Faso,2021,8,1,rural,0.0,Unimproved,5,indoors: latrine to septic tank,no,yes,no,no,2.0,male,41,poorer,"no education, preschool",married,unimproved
16113,Burkina Faso,2021,6,1,rural,0.0,Unimproved,10,no facility/outdoors/bush,no,no,no,no,2.0,male,66,poorest,"no education, preschool",married,unimproved


In [80]:
print(df_sel['distance_to_water_source'].unique())

['20' '30' '5' '10' '15' '60' '40' 'on premises' '45' '35' '90'
 "don't know" '6' '25' '7' '3' '120' '330' '2' '1' '50' '8' '4' '12' '9'
 '70' '0' '160' '100' '11' '32' '300' '18' '140' '14' '65' '240' '600'
 '80' '180' '150' '75' '420' '250' '360' '130' '200' '55' '230' '400' '37'
 '220' '320' '110' '500' '162' '355' '36' '800' '700' '190' '720' '16'
 '28' '17' '22' '13' '54' '135' '23' '260' '350' '19' '900' '26' '210'
 '145' '480' '228' '21' '34' '115' '610' '156' '170' '105' '27' '85' '345'
 '95' '125' '24' '58' '48' '84' '38' '59' '52' '33' '66' '67' '108' '39'
 '56' '760' '42' '440' '44' '68' '340' '43' '270' '53' '63' '31' '665'
 '62' '29' '430' '540' '660' '503' '840' '780' '144' '380' '146' '72'
 '450' '46' '425' '155' '310' '215' '280' '136' '185' '122' '64' '41'
 '165' '118' '650' '630' '123' '390' '410' '460' '490' '995' '111' '61'
 '69' '152' '640' '71' '57' '520' '96' '750' '370' '148' '710' '242' '290'
 '128' '455' '560' '175' '815' '465' '49' '365' '104' '126' '132' '24

In [81]:
import numpy as np
import pandas as pd

# Handle specific string values before converting to numeric
# Using .loc to prevent SettingWithCopyWarning if df_sel is a view
# If 'distance_to_water_source' is already numeric (from a previous run), these .loc replacements will not find matches.
df_sel.loc[df_sel['distance_to_water_source'] == 'on premises', 'distance_to_water_source'] = '0'
df_sel.loc[df_sel['distance_to_water_source'] == "don't know", 'distance_to_water_source'] = '31'

# Convert 'distance_to_water_source' to numeric, coercing any other errors to NaN
df_sel['distance_to_water_source'] = pd.to_numeric(df_sel['distance_to_water_source'], errors='coerce')

# Categorize 'distance_to_water_source' into two groups: '<=30' and '>30'
# Missing values will remain pd.NA in the new category column.
df_sel['distance_category'] = np.where(
    df_sel['distance_to_water_source'].isna(),
    pd.NA, # Use pd.NA instead of np.nan for compatibility with string types
    np.where(df_sel['distance_to_water_source'] <= 30, '<=30', '>30')
)

print(df_sel[['distance_to_water_source', 'distance_category']].head())
print("\nValue counts for distance_category:")
print(df_sel['distance_category'].value_counts(dropna=False))

       distance_to_water_source distance_category
16109                      20.0              <=30
16110                      30.0              <=30
16111                       5.0              <=30
16112                       5.0              <=30
16113                      10.0              <=30

Value counts for distance_category:
distance_category
<=30    380486
>30      85803
<NA>        26
Name: count, dtype: int64


/tmp/ipython-input-3980812253.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['distance_to_water_source'] = pd.to_numeric(df_sel['distance_to_water_source'], errors='coerce')
/tmp/ipython-input-3980812253.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['distance_category'] = np.where(


In [82]:
df_sel.head()

,country_code_phase,interview_year,Number_household_members,Number_children_under5,Residence,total_adults,source_drinking_water,distance_to_water_source,toilet_type,Has_electricity,...,Has_television,Has_refrigerator,Numer_sleeping_rooms,household_head_sex,Household_head_age,Household_wealth_index,Household_education_level,Household_head_marital_status,toilet_types,distance_category
16109,Burkina Faso,2021,4,1,rural,1.0,Unimproved,20.0,no facility/outdoors/bush,no,...,no,no,2.0,male,32,poorest,primary,married,unimproved,<=30
16110,Burkina Faso,2021,7,2,rural,0.0,Unimproved,30.0,no facility/outdoors/bush,no,...,no,no,2.0,male,34,middle,"no education, preschool",married,unimproved,<=30
16111,Burkina Faso,2021,9,2,rural,1.0,Unimproved,5.0,no facility/outdoors/bush,no,...,no,no,4.0,male,34,poorest,secondary,married,unimproved,<=30
16112,Burkina Faso,2021,8,1,rural,0.0,Unimproved,5.0,indoors: latrine to septic tank,no,...,no,no,2.0,male,41,poorer,"no education, preschool",married,unimproved,<=30
16113,Burkina Faso,2021,6,1,rural,0.0,Unimproved,10.0,no facility/outdoors/bush,no,...,no,no,2.0,male,66,poorest,"no education, preschool",married,unimproved,<=30


In [83]:
print(df_sel['distance_category'].value_counts(normalize=True, dropna=False))

distance_category
<=30    0.815942
>30     0.184002
<NA>    0.000056
Name: proportion, dtype: float64


In [84]:
df_sel.columns

Index(['country_code_phase', 'interview_year', 'Number_household_members',
       'Number_children_under5', 'Residence', 'total_adults',
       'source_drinking_water', 'distance_to_water_source', 'toilet_type',
       'Has_electricity', 'Has_radio', 'Has_television', 'Has_refrigerator',
       'Numer_sleeping_rooms', 'household_head_sex', 'Household_head_age',
       'Household_wealth_index', 'Household_education_level',
       'Household_head_marital_status', 'toilet_types', 'distance_category'],
      dtype='object')

In [85]:
import numpy as np

df_sel['household_members_category'] = np.where(
    df_sel['Number_household_members'] <= 6,
    '1-6',
    '>6'
)

print(df_sel[['Number_household_members', 'household_members_category']].head())
print("\nValue counts for household_members_category:")
print(df_sel['household_members_category'].value_counts(dropna=False))

       Number_household_members household_members_category
16109                         4                        1-6
16110                         7                         >6
16111                         9                         >6
16112                         8                         >6
16113                         6                        1-6

Value counts for household_members_category:
household_members_category
1-6    353093
>6     113222
Name: count, dtype: int64


/tmp/ipython-input-1800364442.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['household_members_category'] = np.where(


In [86]:
df_sel.columns

Index(['country_code_phase', 'interview_year', 'Number_household_members',
       'Number_children_under5', 'Residence', 'total_adults',
       'source_drinking_water', 'distance_to_water_source', 'toilet_type',
       'Has_electricity', 'Has_radio', 'Has_television', 'Has_refrigerator',
       'Numer_sleeping_rooms', 'household_head_sex', 'Household_head_age',
       'Household_wealth_index', 'Household_education_level',
       'Household_head_marital_status', 'toilet_types', 'distance_category',
       'household_members_category'],
      dtype='object')

In [87]:
print(df_sel['Numer_sleeping_rooms'].value_counts(dropna=False))

Numer_sleeping_rooms
1.0     165976
2.0     152777
3.0      88210
4.0      33679
5.0      12590
6.0       6088
7.0       2538
8.0       1625
9.0        732
10.0       656
11.0       295
12.0       280
NaN        227
13.0       133
15.0       110
14.0       104
16.0        57
22.0        44
20.0        43
24.0        41
18.0        32
17.0        29
19.0        16
23.0        15
21.0        12
0.0          6
Name: count, dtype: int64


In [89]:
import numpy as np
import pandas as pd # Import pandas to use pd.NA

conditions = [
    df_sel['Numer_sleeping_rooms'] == 1,
    df_sel['Numer_sleeping_rooms'] == 2,
    df_sel['Numer_sleeping_rooms'] >= 3
]

choices = ['1', '2', '>=3']

# Use pd.NA for the default value to ensure type compatibility with string choices
df_sel['sleeping_rooms_category'] = np.select(conditions, choices, default=pd.NA)

print(df_sel[['Numer_sleeping_rooms', 'sleeping_rooms_category']].head())
print("\nValue counts for sleeping_rooms_category:")
print(df_sel['sleeping_rooms_category'].value_counts(dropna=False))

       Numer_sleeping_rooms sleeping_rooms_category
16109                   2.0                       2
16110                   2.0                       2
16111                   4.0                     >=3
16112                   2.0                       2
16113                   2.0                       2

Value counts for sleeping_rooms_category:
sleeping_rooms_category
1       165976
2       152777
>=3     147329
<NA>       233
Name: count, dtype: int64


In [90]:
print(df_sel['Household_head_age'].value_counts(dropna=False))

Household_head_age
35     16124
40     16119
30     15426
45     13617
32     12507
       ...  
14        27
13        16
NaN        8
12         3
11         1
Name: count, Length: 87, dtype: int64


In [92]:
import pandas as pd

# Convert 'Household_head_age' to numeric, coercing errors to NaN
df_sel['Household_head_age'] = pd.to_numeric(df_sel['Household_head_age'], errors='coerce')

# Define the bins and labels for the age categories
bins = [0, 19, 35, 45, df_sel['Household_head_age'].max() + 1]
labels = ['<20', '20-35', '36-45', '>46']

# Recategorize 'Household_head_age' into the new categories
df_sel['Household_head_age_category'] = pd.cut(
    df_sel['Household_head_age'],
    bins=bins,
    labels=labels,
    right=True,  # Bins are (min, max]
    include_lowest=True # Include the lowest value (0)
)

# Display the head of the relevant columns and value counts
print(df_sel[['Household_head_age', 'Household_head_age_category']].head())
print("\nValue counts for Household_head_age_category:")
print(df_sel['Household_head_age_category'].value_counts(dropna=False))

       Household_head_age Household_head_age_category
16109                32.0                       20-35
16110                34.0                       20-35
16111                34.0                       20-35
16112                41.0                       36-45
16113                66.0                         >46

Value counts for Household_head_age_category:
Household_head_age_category
>46      204531
20-35    144897
36-45    109324
<20        5331
NaN        2232
Name: count, dtype: int64


In [94]:
import numpy as np

df_sel['children_under5_category'] = np.where(
    df_sel['Number_children_under5'] <= 2,
    '<=2',
    '>2'
)

print(df_sel[['Number_children_under5', 'children_under5_category']].head())
print("\nValue counts for children_under5_category:")
print(df_sel['children_under5_category'].value_counts(dropna=False))

       Number_children_under5 children_under5_category
16109                       1                      <=2
16110                       2                      <=2
16111                       2                      <=2
16112                       1                      <=2
16113                       1                      <=2

Value counts for children_under5_category:
children_under5_category
<=2    431202
>2      35113
Name: count, dtype: int64


In [95]:
df_sel.columns

Index(['country_code_phase', 'interview_year', 'Number_household_members',
       'Number_children_under5', 'Residence', 'total_adults',
       'source_drinking_water', 'distance_to_water_source', 'toilet_type',
       'Has_electricity', 'Has_radio', 'Has_television', 'Has_refrigerator',
       'Numer_sleeping_rooms', 'household_head_sex', 'Household_head_age',
       'Household_wealth_index', 'Household_education_level',
       'Household_head_marital_status', 'toilet_types', 'distance_category',
       'household_members_category', 'sleeping_rooms_category',
       'Household_head_age_category', 'children_under5_category'],
      dtype='object')

In [96]:
selected_final_columns = [
    'country_code_phase',
    'interview_year',
    'Residence',
    'source_drinking_water',
    'Has_electricity',
    'Has_radio',
    'Has_television',
    'Has_refrigerator',
    'household_head_sex',
    'Household_wealth_index',
    'Household_education_level',
    'Household_head_marital_status',
    'toilet_types',
    'distance_category',
    'household_members_category',
    'sleeping_rooms_category',
    'Household_head_age_category',
    'children_under5_category'
]

df_final = df_sel[selected_final_columns]
display(df_final.head())

,country_code_phase,interview_year,Residence,source_drinking_water,Has_electricity,Has_radio,Has_television,Has_refrigerator,household_head_sex,Household_wealth_index,Household_education_level,Household_head_marital_status,toilet_types,distance_category,household_members_category,sleeping_rooms_category,Household_head_age_category,children_under5_category
16109,Burkina Faso,2021,rural,Unimproved,no,no,no,no,male,poorest,primary,married,unimproved,<=30,1-6,2,20-35,<=2
16110,Burkina Faso,2021,rural,Unimproved,no,yes,no,no,male,middle,"no education, preschool",married,unimproved,<=30,>6,2,20-35,<=2
16111,Burkina Faso,2021,rural,Unimproved,no,no,no,no,male,poorest,secondary,married,unimproved,<=30,>6,>=3,20-35,<=2
16112,Burkina Faso,2021,rural,Unimproved,no,yes,no,no,male,poorer,"no education, preschool",married,unimproved,<=30,>6,2,36-45,<=2
16113,Burkina Faso,2021,rural,Unimproved,no,no,no,no,male,poorest,"no education, preschool",married,unimproved,<=30,1-6,2,>46,<=2


In [98]:
missing_values_percent_final = df_final.isnull().sum() / len(df_final) * 100
print("Percentage of missing values per column in df_final:")
print(missing_values_percent_final[missing_values_percent_final > 0].sort_values(ascending=False))

Percentage of missing values per column in df_final:
Household_head_marital_status    2.012588
Household_head_age_category      0.478646
sleeping_rooms_category          0.049966
Household_education_level        0.028736
toilet_types                     0.018442
Has_refrigerator                 0.006648
distance_category                0.005576
Has_electricity                  0.004932
Has_television                   0.004503
Has_radio                        0.003217
dtype: float64


In [99]:
for col in df_final.columns:
    if df_final[col].isnull().any():
        mode_value = df_final[col].mode()[0]  # Get the first mode if there are multiple
        df_final[col].fillna(mode_value, inplace=True)

print("Missing values after mode imputation:")
print(df_final.isnull().sum())

/tmp/ipython-input-2064138365.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final[col].fillna(mode_value, inplace=True)
/tmp/ipython-input-2064138365.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final[col].fillna(mode_value, inplace=True)


Missing values after mode imputation:
country_code_phase               0
interview_year                   0
Residence                        0
source_drinking_water            0
Has_electricity                  0
Has_radio                        0
Has_television                   0
Has_refrigerator                 0
household_head_sex               0
Household_wealth_index           0
Household_education_level        0
Household_head_marital_status    0
toilet_types                     0
distance_category                0
household_members_category       0
sleeping_rooms_category          0
Household_head_age_category      0
children_under5_category         0
dtype: int64


In [101]:
df_final.head()

,country_code_phase,interview_year,Residence,source_drinking_water,Has_electricity,Has_radio,Has_television,Has_refrigerator,household_head_sex,Household_wealth_index,Household_education_level,Household_head_marital_status,toilet_types,distance_category,household_members_category,sleeping_rooms_category,Household_head_age_category,children_under5_category
16109,Burkina Faso,2021,rural,Unimproved,no,no,no,no,male,poorest,primary,married,unimproved,<=30,1-6,2,20-35,<=2
16110,Burkina Faso,2021,rural,Unimproved,no,yes,no,no,male,middle,"no education, preschool",married,unimproved,<=30,>6,2,20-35,<=2
16111,Burkina Faso,2021,rural,Unimproved,no,no,no,no,male,poorest,secondary,married,unimproved,<=30,>6,>=3,20-35,<=2
16112,Burkina Faso,2021,rural,Unimproved,no,yes,no,no,male,poorer,"no education, preschool",married,unimproved,<=30,>6,2,36-45,<=2
16113,Burkina Faso,2021,rural,Unimproved,no,no,no,no,male,poorest,"no education, preschool",married,unimproved,<=30,1-6,2,>46,<=2


In [102]:
for column in df_final.columns:
    print(f"\nProportions for '{column}':")
    print(df_final[column].value_counts(normalize=True, dropna=False))



Proportions for 'country_code_phase':
country_code_phase
Nigeria                      0.085880
Kenya                        0.081299
Malawi                       0.056530
Congo Democratic Republic    0.056500
Madagascar                   0.043983
Uganda                       0.042006
Ghana                        0.038457
Chad                         0.036956
Angola                       0.034833
Burundi                      0.034262
Tanzania                     0.033679
Côte d’Ivoire                0.031665
Mozambique                   0.030559
Benin                        0.030357
Sierra Leone                 0.028734
Burkina Faso                 0.028416
Rwanda                       0.027769
Zambia                       0.027516
Gabon                        0.025264
Cameroon                     0.025112
Mauritania                   0.025000
South Africa                 0.023767
Zimbabwe                     0.022590
Lesotho                      0.021037
Mali                         0